Crawler for Faculty of Science (Type 3)
---

# 1. Metatable containing all Faculty of Science courses
We will be using Selenium to crawl the course data. This is because the target website is dynamic, so the crawler would have to click a few buttons before arriving at the page containing all the course information.

## Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# import selenium related packages
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [3]:
# define webdriver path
CHROME_DRIVER_PATH = './drivers/chromedriver'

In [4]:
# define url to be crawled
URL = 'https://webapp.science.hku.hk/sr4/servlet/enquiry?frmid=MenuC'

## Helper functions

In [5]:
# helper function
# launch chrome browser (headless)
# return: BROWSER type
def launch_chrome_browser(CHROME_DRIVER_PATH):
    CHROME_OPTIONS = Options()
    CHROME_OPTIONS.add_argument('--headless')
    browser = webdriver.Chrome(CHROME_DRIVER_PATH, options=CHROME_OPTIONS)
    return browser

In [6]:
# helper function
# click on 'Search' button
def click_button(browser):
    # click on 'Search' button to get to all course information
    btn = browser.find_element_by_xpath("//input[contains(@name, 'Search')]")
    btn.click()

In [7]:
# helper function
# get information of all Faculty of Science courses
# return DATAFRAME
def get_courses(browser):
    # get big table with all the course information
    course_table = browser.find_element_by_xpath("//table[contains(@class, 'courseList')]")

    # get every row of the big table
    rows = course_table.find_elements_by_xpath("//tr")

    # define the matrix
    courses = []

    # iterate through every row to generate the final dataframe
    for row in rows:
        cols = row.find_elements_by_tag_name('td')

        # only append the course data for rows with 16 cols
        if (len(cols) == 16):
            info = []

            for col in cols:
                info.append(col.text)
            courses.append(info)
        
        
    df_courses = pd.DataFrame(
        courses,
        columns=[
            'No.', 'Course Code', 'Title', 'Credit', 'Pre-requisite', 'Available in 2022-2023',
            'Available in 2023-2024', 'Semester offered in 2022-2023', 'Exam held in 2022-2023', 
            'Quota', 'Course Coordinator', 'Disciplinary Core Course', 'Disciplinary Elective', 
            'Capstone - Disciplinary Core Course', 'Capstone - Disciplinary Elective', 'Teachers'
        ]
    )
        
    return df_courses

## Generate actual course data using Selenium

In [8]:
# launch chrome browser
browser = launch_chrome_browser(CHROME_DRIVER_PATH)

# launch website
browser.get(URL)
sleep(2)

# click on 'Search' button to get to the page with all course information
click_button(browser)

# generate the dataframe containing all Faculty of Science courses
df_courses = get_courses(browser)

In [21]:
# save the dataframe to csv
df_courses.to_csv('data/df_science_courses_pre.csv')

But since each of these courses may have multiple professors, we also need to generate another dataframe with the course code and the professors. For each course, we may have multiple rows each containing a different professor. The primary key here is 'Course Code'.

In [10]:
# create a subset
df_courses_and_professors = df_courses[['No.', 'Course Code', 'Title', 'Teachers']]

df_courses_and_professors

,No.,Course Code,Title,Teachers
0,1,BIOC1600,Perspectives in biochemistry,"(Dr B C W Wong,Biomedical Sciences)\n(Dr B H B..."
1,2,BIOC2600,Basic biochemistry,"(Dr A S L Wong,Biomedical Sciences)\n(Dr C M Q..."
2,3,BIOC3601,Basic metabolism,"(Dr B H B Yuen,Biomedical Sciences)\n(Dr L W L..."
3,4,BIOC3604,Essential techniques in biochemistry and molec...,"(Dr R C C Chang,Biomedical Sciences)\n(Dr B C ..."
4,5,BIOC3605,Sequence bioinformatics,"(Dr B C W Wong,Biomedical Sciences)\n(Dr J W K..."
...,...,...,...,...
346,347,CCST9051,What are We Made of - the Fundamental Nature o...,
347,348,CCST9054,"War, Peace, and the Natural World",
348,349,CCST9056,The Force is with You: How Things Work,
349,350,CCST9067,Leaving Earth: Our Future in Space,


In [11]:
# convert the 'Teachers' column to a list instead of string
df_courses_and_professors['Teachers'] = df_courses_and_professors['Teachers'].map(lambda x: x.split('\n'))

c:\Users\Kackie\anaconda3\envs\DataScience\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
df_courses_and_professors['Teachers']

0      [(Dr B C W Wong,Biomedical Sciences), (Dr B H ...
1      [(Dr A S L Wong,Biomedical Sciences), (Dr C M ...
2      [(Dr B H B Yuen,Biomedical Sciences), (Dr L W ...
3      [(Dr R C C Chang,Biomedical Sciences), (Dr B C...
4      [(Dr B C W Wong,Biomedical Sciences), (Dr J W ...
                             ...                        
346                                                   []
347                                                   []
348                                                   []
349                                                   []
350                                                   []
Name: Teachers, Length: 351, dtype: object

In [13]:
# use the explode() function to generate new row for any course with multiple teachers
df_courses_and_professors = df_courses_and_professors.explode('Teachers')

In [14]:
df_courses_and_professors.head()

,No.,Course Code,Title,Teachers
0,1,BIOC1600,Perspectives in biochemistry,"(Dr B C W Wong,Biomedical Sciences)"
0,1,BIOC1600,Perspectives in biochemistry,"(Dr B H B Yuen,Biomedical Sciences)"
0,1,BIOC1600,Perspectives in biochemistry,"(Dr. J W Y Ho,Biomedical Sciences)"
0,1,BIOC1600,Perspectives in biochemistry,"(Dr. M S Y Huen,Biomedical Sciences)"
0,1,BIOC1600,Perspectives in biochemistry,"(Dr. Y S Chan,Department of Paediatrics and Ad..."


In [15]:
# notice that for the 'Teachers' column we have a pair of the teacher's name and department, so we convert that into a tuple
df_courses_and_professors['Teachers'] = df_courses_and_professors['Teachers'].map(lambda x: tuple(x.lstrip('(').rstrip(')').split(',')))

In [16]:
df_courses_and_professors.head()

,No.,Course Code,Title,Teachers
0,1,BIOC1600,Perspectives in biochemistry,"(Dr B C W Wong, Biomedical Sciences)"
0,1,BIOC1600,Perspectives in biochemistry,"(Dr B H B Yuen, Biomedical Sciences)"
0,1,BIOC1600,Perspectives in biochemistry,"(Dr. J W Y Ho, Biomedical Sciences)"
0,1,BIOC1600,Perspectives in biochemistry,"(Dr. M S Y Huen, Biomedical Sciences)"
0,1,BIOC1600,Perspectives in biochemistry,"(Dr. Y S Chan, Department of Paediatrics and A..."


In [17]:
# put the first value in the tuple under a new 'Teacher' column, and the second value in the tuple under a new 'Teacher Department' column
df_courses_and_professors['Teacher'] = df_courses_and_professors['Teachers'].map(lambda x: x[0])
df_courses_and_professors['Teacher Department'] = df_courses_and_professors['Teachers'].map(lambda x: x[-1])

In [18]:
# finally delete the old 'Teachers' column
df_courses_and_professors = df_courses_and_professors.drop(columns=['Teachers'])

In [22]:
# save to csv
df_courses_and_professors.to_csv('data/df_science_courses_and_professors.csv')

# 2. Individual Course Details Page
For the detailed course descriptions, we need to crawl the individual course details pages based on the course URL. An example of the URL is https://webapp.science.hku.hk/sr4/servlet/enquiry?Type=Course&course_code=BIOC1600. In particular, the Beautiful Soup crawler will access all the courses stored in our previous 'df_courses_pre.csv' dataset, and go through each course page using the URL.

In [23]:
# read the saved 'df_courses_pre.csv' dataset
df_courses = pd.read_csv('data/df_science_courses_pre.csv', index_col=0)

In [24]:
# select the 'Course Code' column
course_codes = df_courses['Course Code']

In [25]:
course_codes

0      BIOC1600
1      BIOC2600
2      BIOC3601
3      BIOC3604
4      BIOC3605
         ...   
346    CCST9051
347    CCST9054
348    CCST9056
349    CCST9067
350    CCST9068
Name: Course Code, Length: 351, dtype: object

## Helper functions

In [26]:
from bs4 import BeautifulSoup
from time import sleep
import requests

In [27]:
# helper function
# generate course details page URL
# return: STRING
def generate_course_url(course_code):
    url = 'https://webapp.science.hku.hk/sr4/servlet/enquiry?Type=Course&course_code=' + course_code
    return url

In [28]:
# helper function
# get the table that contains all the course details
# return: BeautifulSoup format
def get_course_table(course_code):
    # get url based on course_code
    url = generate_course_url(course_code)

    # set up beautiful soup configurations
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # find the table containing the table
    table = soup.find('table', {'class': 'courseDetails'})

    return table

In [29]:
# helper function
# return: DATAFRAME
def get_course_details(course_code):
    table = get_course_table(course_code)

    # find all rows inside this course details table
    trs = table.find_all('tr')

    # define lists for headers and values
    headers = []
    values = []

    for tr in trs:
        try:
            header = tr.find('th').text.strip()
            value = tr.find('td').text.strip().replace('\n', ' ').replace('\t', ' ')

            if header == 'Course Learning Outcomes':
                headers.append(header)

                # get learning_outcomes
                learning_outcomes = tr.find_all('tr')
                learning_outcomes = list(map(lambda x: x.find_all('td')[1].text.strip().replace('\n', ': '), learning_outcomes))
                learning_outcomes = '; '.join(learning_outcomes)

                # append learning_outcomes
                values.append(learning_outcomes)

            elif header == 'Pre-requisites (and Co-requisites and Impermissible combinations)':
                headers.append(header)
                value = tr.find('td').get_text(strip=True, separator='; ')
                values.append(value)

            elif header == 'Course Status with Related Major/Minor /Professional Core':
                headers.append(header)
                value = tr.find('td').get_text(strip=True, separator='; ').replace('\r', '').replace('\n', '')
                values.append(value)

            elif header == 'Course Teaching & Learning Activities':
                # get various types of learning activities
                activities = tr.find_all('tr')

                # get activities values
                activities_values = list(map(lambda x: x.find('td', {'class': 'right'}), activities))
                activities_values = list(filter(None, activities_values))
                activities_values = list(map(lambda x: x.text.strip(), activities_values))
                
                # get activities names
                activities = list(map(lambda x: x.find('td'), activities))
                activities = list(filter(None, activities)) # remove NoneType elements in the list
                activities = list(map(lambda x: x.text.strip() + ' No. of Hours', activities))

                headers.extend(activities)
                values.extend(activities_values)

            elif header == 'Assessment Methods and Weighting':
                # get various types of assmessments
                assessments = tr.find_all('tr')

                # get assessment weightings
                weightings = list(map(lambda x: x.find('td', {'class': 'right'}), assessments))
                weightings = list(filter(None, weightings))
                weightings = list(map(lambda x: x.text.strip(), weightings))

                # get assessment types
                types = list(map(lambda x: x.find('td'), assessments))
                types = list(filter(None, types))
                types = list(map(lambda x: x.text.strip() + ' Weighting in final course grade (%)', types))

                headers.extend(types)
                values.extend(weightings)

            else:
                headers.append(header)
                values.append(value)
        except: 
            continue

    df = pd.DataFrame([values], columns = headers)
    df.insert(0, 'Course Code', course_code)
    
    return df 

In [30]:
# helper function 
# go through every course and get the details
# return: DATAFRAME
def get_all_course_details(course_codes):
    df = pd.DataFrame()
    for course_code in course_codes:
        df_course = get_course_details(course_code)
        df = df.append(df_course)
         
    return df

## Get all course details

In [31]:
df_all_course_details = get_all_course_details(course_codes)

In [32]:
# reset index
df_all_course_details = df_all_course_details.reset_index(drop = True)

In [33]:
# save all course details to csv
df_all_course_details.to_csv('data/df_science_all_course_details.csv')